In [78]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from termcolor import colored
import time

import tensorflow as tf
import winsound

In [79]:
model = tf.keras.models.load_model('clf.keras')

In [80]:
def concatenate_factuals_csv(directory):
    dfs = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('factuals.csv'):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                dfs.append(df)
    
    if dfs:
        concatenated_df = pd.concat(dfs, ignore_index=True)
        return concatenated_df
    else:
        print("No 'factuals.csv' files found.")
        return pd.DataFrame()  # Return an empty DataFrame if no files found


directory = 'Y:/Data/factuals samples from Mayo data'

all_factuals = concatenate_factuals_csv(directory)

In [81]:
all_factuals.sample(frac = 1).head()

,s_id,Age,Gender,Ethnicity,A1C,CarbSize,TotalBolus,del_t,Mode,TotalBasal,PreMeal_BGL_slope,PreMeal_BGL,Max_BGL
630,36,32,F,Hispanic,5.0,15.0,4.18,5,0,0.538917,-0.257143,133.0,150.0
329,19,66,F,White,7.2,35.0,7.50,35,0,0.551500,0.342857,118.8,213.6
583,36,32,F,Hispanic,5.0,25.0,12.50,40,0,1.050000,-2.914286,102.0,143.0
683,36,32,F,Hispanic,5.0,35.0,0.00,70,0,0.450000,6.828571,176.0,137.0
602,36,32,F,Hispanic,5.0,15.0,0.00,70,0,0.779333,1.228571,97.0,141.0


In [82]:
s_id = [9,10,15,18,21,25,29,43,76]
dir = 'C:/Users/Tech Land/Desktop/factuals samples from Mayo data'
dfs = []


for id in s_id:
    file_path = dir + '/' + str(id) + '_factuals.csv'
    df = pd.read_csv(file_path)
    dfs.append(df)

concatenated_df = pd.concat(dfs, ignore_index=True)
all_factuals = pd.concat((all_factuals,
                    concatenated_df.loc[np.random.choice(np.where(concatenated_df['Max_BGL']>180)[0], size=600, replace=False)],
                    concatenated_df.loc[np.random.choice(np.where(concatenated_df['Max_BGL']<=180)[0], size=100, replace=False)]), 
                    ignore_index=True)

In [83]:
all_factuals['label'] = np.zeros(all_factuals.shape[0])
all_factuals.loc[np.where(all_factuals['Max_BGL']>180)[0],'label'] = 1

all_factuals['Mode'] = all_factuals['Mode'].replace({'sleep': 1, 'exercise': 2})
all_factuals['Gender'] = all_factuals['Gender'].replace({'F': 0, 'M': 1})
all_factuals['Ethnicity'] = all_factuals['Ethnicity'].replace({'White': 0, 'Hispanic': 1})

all_factuals = all_factuals.apply(pd.to_numeric, errors='coerce')
all_factuals = all_factuals.dropna()

In [84]:
data = pd.read_csv('Mayo_data.csv')
X = data.loc[:,:'PreMeal_BGL'].to_numpy()
y = data.loc[:,'label'].to_numpy()

In [85]:
_, X_test, _, y_test = train_test_split(X, y, test_size = 0.15, random_state=1)
y_test = to_categorical(y_test)

In [86]:
def create_min_max_dataframe(factuals):
    unique_persons = factuals.drop_duplicates(subset=['s_id','Age', 'Gender', 'Ethnicity', 'A1C'], keep='first').copy().sort_values(by='s_id')
    num_cols = factuals.columns.drop(['s_id','Max_BGL','label'])

    for col in num_cols:
        unique_persons.loc[:,f'{col}_min'] = factuals.groupby(['s_id','Age', 'Gender', 'Ethnicity', 'A1C'])[col].min().to_numpy()
        unique_persons.loc[:,f'{col}_max'] = factuals.groupby(['s_id','Age', 'Gender', 'Ethnicity', 'A1C'])[col].max().to_numpy()
    unique_persons.drop(['CarbSize', 'TotalBolus', 'del_t','Mode', 'TotalBasal', 'PreMeal_BGL_slope', 'PreMeal_BGL', 'Max_BGL', 'label'],axis=1,inplace=True)
    return unique_persons
unique_persons = create_min_max_dataframe(all_factuals)

In [87]:
def calculate_saliency_scores(model, x_prime, target_class, modifiable_features_indices, δ):
    S = np.zeros(x_prime.shape[1])
    for idx in modifiable_features_indices:
        x_prime_plus = np.copy(x_prime)
        x_prime_plus[0][idx] += δ[modifiable_features_indices.index(idx)]
        S[idx] = (model.predict(x_prime_plus)[0][target_class] - model.predict(x_prime)[0][target_class]) / δ[modifiable_features_indices.index(idx)]
    return S


In [88]:
def normalize_scores(S):
    valid_scores = S[S != 0]
    if valid_scores.size > 0:
        normalized = (valid_scores - valid_scores.min()) / (valid_scores.max() - valid_scores.min())
        positioned = np.zeros_like(S)
        positioned[S != 0] = normalized
        return positioned
    return S


In [89]:
def compute_combined_scores(S, feature_weights, user_prefs, multiplier, modifiable_features_indices):
    combined_scores = np.zeros_like(S)
    for idx in modifiable_features_indices:
        modifiable_idx = modifiable_features_indices.index(idx)
        combined_scores[idx] = (S[idx] + (feature_weights[modifiable_idx] + user_prefs[modifiable_idx])) * multiplier[modifiable_idx]
    return combined_scores


In [90]:
def select_best_feature(P, combined_scores, modifiable_features_indices, previous_feature, n):
    max_score = -np.inf
    best_feature = -1
    for idx in modifiable_features_indices:
        if P[idx] < n and combined_scores[idx] > max_score and idx != previous_feature:
            max_score = combined_scores[idx]
            best_feature = idx
    return best_feature


In [91]:
def update_feature(x_prime, i, S, δ, modifiable_features_indices, feature_min, feature_max, multiplier):
    x_prime[0][i] += np.sign(S[i]) * δ[modifiable_features_indices.index(i)]
    if x_prime[0][i] < feature_min[i]:
        x_prime[0][i] = feature_min[i]
        multiplier[modifiable_features_indices.index(i)] = 0
    elif x_prime[0][i] > feature_max[i]:
        x_prime[0][i] = feature_max[i]
        multiplier[modifiable_features_indices.index(i)] = 0
    return x_prime


In [92]:
def generate_counterfactual_with_preferences(model, x, target_class, N, γ, δ, n, modifiable_features_indices, feature_min, feature_max, feature_weights, user_prefs, multiplier, patience=2):
    x_prime = np.copy(x)
    c = 0
    no_change_count = 0
    previous_feature = None
    np_vector = len(x[0])
    P = np.zeros(np_vector)

    while model.predict(x_prime)[0][target_class] < γ and c < N:
        # Compute saliency scores
        S = calculate_saliency_scores(model, x_prime, target_class, modifiable_features_indices, δ)

        # Normalize and compute combined scores
        normalized_scores = normalize_scores(S)
        combined_scores = compute_combined_scores(normalized_scores, feature_weights, user_prefs, multiplier, modifiable_features_indices)

        # Select the best feature to modify
        i = select_best_feature(P, combined_scores, modifiable_features_indices, previous_feature, n)
        if i == -1:
            break

        # Save current prediction for comparison
        current_prediction = model.predict(x_prime)[0][target_class]

        # Update the selected feature
        x_prime = update_feature(x_prime, i, S, δ, modifiable_features_indices, feature_min, feature_max, multiplier)
        P[i] += 1  # Track feature modifications
        c += 1

        # Check for significant changes in prediction
        new_prediction = model.predict(x_prime)[0][target_class]
        if abs(new_prediction - current_prediction) < 1e-5:
            no_change_count += 1
        else:
            no_change_count = 0

        # Handle stagnation with patience
        if no_change_count >= patience:
            no_change_count = 0
            previous_feature = i
            S[i] = 0  # Reset saliency score to avoid re-selection

    return x_prime, c

In [93]:
model.predict(X_test[28].reshape(1,11))[0][0] #test sample

1/1 [==============================] - 0s 102ms/step


0.38917288

In [94]:
x_fact = pd.DataFrame(np.array(X_test[28].reshape(1, -1)), columns=all_factuals.columns.tolist()[1:-2])
x_fact['probability'] = model.predict(x_fact)[0][0]
x_fact

1/1 [==============================] - 0s 85ms/step


,Age,Gender,Ethnicity,A1C,CarbSize,TotalBolus,del_t,Mode,TotalBasal,PreMeal_BGL_slope,PreMeal_BGL,probability
0,27.0,1.0,0.0,6.2,50.0,4.21,-45.0,0.0,0.45,2.885714,148.0,0.389173


In [95]:
# Example parameters
x = X_test[28].reshape(1, 11)
target_class = 0
γ = 0.5
N = 400
δ = np.array([5, 0.5, 5, 10])  # Different δ for each modifiable feature
n = 100

# Define the minimum and maximum values for each feature
person = unique_persons.loc[(unique_persons['Age'] == x[0,0]) & (unique_persons['Gender'] == x[0,1]) & (unique_persons['Ethnicity'] == x[0,2]) & (unique_persons['A1C'] == x[0,3])]
feature_min = person[[col for col in person.columns if col.endswith('_min')]].values[0]
feature_max = person[[col for col in person.columns if col.endswith('_max')]].values[0]
feature_min[[0,1,2,3,7,8,9]], feature_max[[0,1,2,3,7,8,9]], feature_max[10]  = 0, 0, 175

modifiable_features_indices = [4, 5, 6, 10]  # Indices for CarbSize, TotalBolus, del_t, TotalBasal, PreMeal_BGL
feature_weights = np.array([1.0, 1.0, 1.0, 1.0])  # Physician-provided feature importance weights
user_prefs = np.array([1.0, 1.0, 1.0, 1.0])  # User preference weights
multiplier = np.array([1.0, 1.0, 1.0, 1.0])

x_prime = generate_counterfactual_with_preferences(model, x, target_class, N, γ, δ, n, modifiable_features_indices, feature_min, feature_max, feature_weights, user_prefs, multiplier)


1/1 [==============================] - 0s 30ms/step


In [96]:
x_p = pd.DataFrame(x_prime[0], columns=all_factuals.columns.tolist()[1:-2])
x_p['probability'] = model.predict(x_prime[0])[0][0]
x_p

1/1 [==============================] - 0s 19ms/step


,Age,Gender,Ethnicity,A1C,CarbSize,TotalBolus,del_t,Mode,TotalBasal,PreMeal_BGL_slope,PreMeal_BGL,probability
0,27.0,1.0,0.0,6.2,40.0,4.21,-45.0,0.0,0.45,2.885714,128.0,0.503511


In [97]:
x_fact

,Age,Gender,Ethnicity,A1C,CarbSize,TotalBolus,del_t,Mode,TotalBasal,PreMeal_BGL_slope,PreMeal_BGL,probability
0,27.0,1.0,0.0,6.2,50.0,4.21,-45.0,0.0,0.45,2.885714,148.0,0.389173
